In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')


1. Yes24 베스트셀러 리스트 크롤링(기본 방식)(지난시간과 동일함 그대로임)

In [ ]:
# 필요한 라이브러리 불러오기
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 크롤링할 페이지의 URL을 지정
URL = 'https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=1&pageSize=24'  #너 어느 사이트에 가야하는지 타깃을 정해주는 것이 먼저이다.

# 웹 페이지 요청: 지정된 URL에 HTTP GET 요청을 보내고, 응답 데이터를 html에 저장
html = requests.get(URL)  #물론 URL을 변수화하지 않고 그대로 넣어도 된다.
soup = BeautifulSoup(html.text, 'html.parser') #파서는 우리가 자료를 정리할 때 사용하라고 주는 도구이다!

# 'itemUnit' 클래스가 있는 모든 div 선택
books = soup.find_all('div', class_='itemUnit')

titles = []
for book in books:
    title = book.find('a', class_='gd_name').text
    titles.append(title)
# print(titles)

# Pandas DataFrame으로 변환 #위에서 가져온 리스트를 데이터 분석을 위해 데이터프레임으로 변형한다
df = pd.DataFrame(titles, columns=["책 제목"])  #칼럼명이 원래 없지만, 칼럼명을 설정하도록 하자!
df = pd.DataFrame(titles) #이것도 가능하지만, 칼럼명은 안나온다. 즉, 칼럼명 우리가 원하면 추가할 수 있는 것이다.

# CSV 파일로 저장
csv_path = "/content/drive/MyDrive/mypython/myweb/yes24_bestsellers_titles.csv"
df.to_csv(csv_path, index=False, encoding='utf-8-sig')

# 엑셀 파일로 저장
excel_path = "/content/drive/MyDrive/mypython/myweb/yes24_bestsellers_titles.xlsx"
df.to_excel(excel_path, index=False)

#리퀘스트 패키지: 원본 html을 가져오는 라이브러이이다.
#뷰티풀수프 패키지: 원본 html 코드를 추출할 수 있도록 변환시켜주는 라이브러리이다. 이 두 가지가 웹 크롤링 할 때 꼭 필요한 앱이다

2. Yes24 베스트셀러 리스트 크롤링(실전활용코드)
* 헤더(headers) 설정: User-Agent를 넣어 웹 브라우저처럼 보이게 함(차단이 강한 사이트에서 필수)
* `raise_for_status()` 오류 처리(요청 실패 시 바로 에러 발생 - 디버깅 편리)
* 강력한 파서 `lxml` 사용: HTML 구조가 복잡한 사이트일수록
html.parser보다 lxml이 안정적이고 빠름(요즘 쓰는 새로운 파서!)
* CSS 선택자 활용 select.get_text(strip=True) 혹은 select_one.get_text(strip=True)로 텍스트만 깔끔하게 추출(find all 대신 이 코드를 쓴다)
* for문 기반 반복 수집 구조: 실무 크롤링 코드의 기본 패턴
(요즘은 사이트가 보안을 걸어둔다. 봇으로 하도 해킹하니까... 그래서 봇이 아닌 것처럼 사이트를 속여야 한다)

In [ ]:
# 필요한 라이브러리 불러오기
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 크롤링할 페이지의 URL을 지정
URL = "https://www.yes24.com/Product/Category/BestSeller?categoryNumber=001&pageNumber=1&pageSize=24"

# 웹 브라우저에서 접근하는 것처럼 보이게 하는 헤더 설정
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36",
    "Referer": URL
} #이게 무슨 뜻인지 몰라도 된다. 그냥 붙이는 것이다
# 한 마디로 속이는 것이다. 보안이 많이 걸려있는 곳에.. ->헤더설정이다
# 웹 페이지 요청: 지정된 URL에 HTTP GET 요청을 보내고, 응답 데이터를 html에 저장

html = requests.get(URL, headers=headers) # 이전이랑 똑같은데 headers=headers 생겼다. 즉, 허락을 받을 때 헤더를 넣어줘서 저 봇 아니에요.. 이렇게 하는 것이다
html.raise_for_status() # 요청이 실패하면 에러 발생: 차단가능성이 높은 사이트에서 사용 #그냥 넣는 것이다 해석할 필요 없다.

soup = BeautifulSoup(html.text, "lxml")  # 또는 "html.parser" 이전에 파서 열쇠보다 더 추출이 잘 되는 열쇠인 lmxl를 넣은 것이다.

# 'itemUnit' 클래스가 있는 모든 div 선택
books = soup.select("div.itemUnit") # books = soup.find_all('div', class_='itemUnit')   이전에는 이거였다. 모양은 다르지만, 똑같은 내용이다.
#모든 클래스 이름이 itmeUnit인 div 다 가져와!!

# 각 책 제목 추출
titles = []
for book in books:
    title = book.select_one("a.gd_name").get_text(strip=True)  #여기는 추가로 get_text를 썼다. select_one(): CSS 선택자로 태그 찾기
    titles.append(title) # .get_text() = 텍스트만 추출(태그 제거)strip=True는 문자열 양쪽 공백 제거, 줄바꿈 제거
# 둘이 다르다
titles = []
for book in books:
    title = book.find('a', class_='gd_name').text.strip()
    titles.append(title)
# print(titles)

# print(titles)   # 이 부분도 기존과 다르다.
# 문제 예시: 다음 중 파서가 아닌 것은? 파서는 "html.parser" / "lxml" 두개이다. 아닌거 하나 고르면 된다.

# Pandas DataFrame으로 변환
df = pd.DataFrame(titles, columns=["서적명"])

# CSV 파일로 저장 (Drive의 mypython 폴더 안에 저장)
csv_path = "/content/drive/MyDrive/mypython/myweb/yes24_베스트셀러_서적명.csv"
df.to_csv(csv_path, index=False, encoding='utf-8-sig')

# 엑셀 파일로 저장
excel_path = "/content/drive/MyDrive/mypython/myweb/yes24_베스트셀러_서적명.xlsx"
df.to_excel(excel_path, index=False)

3. 멜론음원차트 TOP100 리스트 크롤링(실전활용코드)
[https://www.melon.com/chart/index.htm](https://www.melon.com/chart/index.htm)

* 아래 코드는 **실전 웹 크롤링에서 반드시 필요한 요소들**을 모두 포함한 완성형 예시임  
* Melon은 크롤링 차단이 강하기 때문에, 학습용이 아닌 **실무 구조에 가까운 크롤러**를 소개함.





In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 크롤링할 Melon TOP100 차트 페이지의 URL을 지정
URL = "https://www.melon.com/chart/index.htm"

# 웹 브라우저에서 접근하는 것처럼 보이게 하는 헤더 설정
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0 Safari/537.36",
    "Referer": "https://www.melon.com/chart/"
}

# 웹 페이지 요청: 지정된 URL에 HTTP GET 요청을 보내고, 응답 데이터를 res에 저장
html = requests.get(URL, headers=headers)
html.raise_for_status()  # 요청이 실패하면 에러 발생: Melon은 차단가능성이 높음

# HTML 코드 파싱: 받은 HTML 코드를 분석 가능한 형태로 변환해서 원하는 데이터를 탐색하고 추출
soup = BeautifulSoup(html.text, "lxml")  # 'lxml': HTML 코드를 분석하기 위한 더 강력한 파서

# 데이터 추출: 차트의 곡 정보를 담을 리스트 data 생성
data = []

# TOP100 곡들이 있는 행(tr 태그 = 태이블행 태그) 중 클래스가 lst50 또는 lst100인 요소를 모두 선택
# Melon은 복잡한 표 구조라 CSS 선택자(select, select_one)로 여러 정보(순위·제목·가수·앨범)를 함께 추출
# Melon TOP100 차트에서 1~50위(lst50)와 51~100위(lst100) 곡 정보가 들어 있는 행을 모두 가져오는 것
# HTML에서 <tr> 태그 중 클래스가 lst50 또는 lst100인 모든 행(row) 요소를 선택
rows = soup.select("tr.lst50, tr.lst100")
#우리가 select 할 때 클래스 이름이 lst50인 tr태그 다 가져와 그리고 클래스 네임이 lst100인 tr태그도 다 가져와!! 이런 의미이다.
#rows에는 꾸러미가 100개가 들어가 있다! 물론, 이걸 들어가보면 정말 어렵게 되어있다.

# 각 곡 정보를 반복해서 추출
for row in rows: #rows 리스트 안의 각 <tr> 요소를 순서대로 반복
    rank = row.select_one("span.rank").get_text(strip=True)  # 순위 추출(현재 행(row)에서 <span> 태그 중 클래스가 rank인 요소를 찾음)
    title = row.select_one("div.wrap_song_info div.rank01 a").get_text(strip=True)  # 곡명 추출(div.wrap_song_info 안에 있는-> div.rank01 안의 -> <a> 태그 선택)
    artist = row.select_one("div.wrap_song_info div.rank02 a").get_text(strip=True)  # 아티스트명 추출(div.wrap_song_info 안에 있는->div.rank02 안의 <a> 태그 선택)
    album = row.select_one("div.wrap_song_info div.rank03 a").get_text(strip=True)  # 앨범명 추출(div.wrap_song_info 안에 있는->div.rank03 안의 <a> 태그 선택)
    #여기서 띄어쓰기로 구분되어 있다.

    # 콘솔에 결과 출력: 순위, 곡명, 아티스트, 앨범 정보를 지정된 형식으로 출력
    print(f"{rank}위: {title} - {artist} (앨범: {album})")

    # 추출한 데이터를 리스트에 추가
    data.append([rank, title, artist, album])

# DataFrame 변환: 추출한 데이터를 판다스 DataFrame 형태로 변환
df = pd.DataFrame(data, columns=["순위", "곡명", "아티스트", "앨범"])

# # 엑셀 저장: DataFrame을 melon_top100.xlsx라는 파일 이름으로 저장
df.to_excel("멜론_100.xlsx", index=False)  #이걸 누르면 옆에 파일창에 뜬다. 이걸 누르면 파일 다운로드가 가능해진다.

# CSV 파일로 저장 (Drive의 mypython 폴더 안에 저장)
csv_path = "/content/drive/MyDrive/mypython/myweb/멜론_100.csv"
df.to_csv(csv_path, index=False, encoding='utf-8-sig')

# 엑셀 파일로 저장
excel_path = "/content/drive/MyDrive/mypython/myweb/멜론_100.xlsx"
df.to_excel(excel_path, index=False)
